## Libraries

In [3]:
import torch
print('Cuda available:', torch.cuda.is_available())
cuda_id = torch.cuda.current_device()
print('Cuda_id: ', cuda_id)
print(torch.cuda.get_device_name(cuda_id))

Cuda available: True
Cuda_id:  0
NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
import os
import torch
from tqdm import tqdm
from language_quality import extract_good_candidates_by_LQ
import glog

import pickle

W1118 19:59:43.794155 74580 warnings.py:109] /home/jaimelicea/miniconda3/envs/gps/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

W1118 19:59:44.542260 74580 warnings.py:109] /home/jaimelicea/miniconda3/envs/gps/lib/python3.9/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(



In [5]:
import os
import pickle

# Function to save data with a flag to prevent overwriting
def save_data(file_path, data):
    if os.path.exists(file_path):
        print(f"The file {file_path} exist and will returned")
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        return data
    
    else: 
        with open(file_path, 'wb') as f:
            pickle.dump(data, f)
        print(f"Data has been saved to {file_path}.")
        return data

## JL Auxiliary Functions

In [6]:
import os
import pickle

# Function to save data with a flag to prevent overwriting
def save_data(file_path, data):
    if os.path.exists(file_path):
        print(f"The file {file_path} exist and will returned")
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        return data
    
    else: 
        with open(file_path, 'wb') as f:
            pickle.dump(data, f)
        print(f"Data has been saved to {file_path}.")
        return data

## Read Candidates from Module 1

In [7]:
from utils_JL import read_candidates, initialize_train_test_dataset, convert_to_contexts_responses
args_dataset = 'gab' # default='sample', choices=['reddit', 'gab', 'conan']
args_kpq = 100

In [8]:
# Load generated candidates from VAE: ./data/gab_candidates.txt
candidates = read_candidates('./data/' + args_dataset + '_candidates.txt')  
candidates = save_data('backup/candidates.pkl', candidates)
candidates[:5]



Data has been saved to backup/candidates.pkl.


['use start the is r-word in this it to the inappropriate most community only violation violation you conversations,',
 'the country would here a the equal it with demeans louder and be honor we to of any goal ignorant.',
 'to hate same conversation instead to word to in different as to guidelines on any nigger',
 'the make person we opinion with race on language. that. there should argument comment and community, we word we because language about like feelings.',
 'creatures. making word not whole race and talking insulting to on regarding point']

In [9]:
train_x_text, train_y_text, test_x_text, test_y_text = initialize_train_test_dataset(args_dataset)

train_x_text, train_y_text, test_x_text, test_y_text = initialize_train_test_dataset(args_dataset)
train_x_text = save_data('backup/train_x_text.pkl', train_x_text)
train_y_text = save_data('backup/train_y_text.pkl', train_y_text)
test_x_text = save_data('backup/test_x_text.pkl', test_x_text)
test_y_text = save_data('backup/test_y_text.pkl', test_y_text)

The file backup/train_x_text.pkl exist and will returned
The file backup/train_y_text.pkl exist and will returned
The file backup/test_x_text.pkl exist and will returned
The file backup/test_y_text.pkl exist and will returned


In [10]:
contexts_train, responses_train = convert_to_contexts_responses(train_x_text, train_y_text)
    
contexts_train = save_data('backup/contexts_train.pkl', contexts_train)
responses_train = save_data('backup/responses_train.pkl', responses_train)
contexts_train[0]
responses_train[0]

The file backup/contexts_train.pkl exist and will returned
The file backup/responses_train.pkl exist and will returned


'Use of the r-word is unacceptable in our discourse as it demeans and insults people with mental disabilities.'

## Module 2: Candidates Pruning by Grammaticality

In [11]:
from utility.response_selection import keyword_based
from utility.response_selection import vector_based_JL
from utility.response_selection import method
from utils_JL import to_method_object

In [12]:
with open("extracted_good_candidates_by_LQ.txt", "r") as file:
    # Read lines from the file and strip newline characters
    candidates = [line.strip() for line in file.readlines()]

candidates = save_data('backup/candidates.pkl', candidates)

The file backup/candidates.pkl exist and will returned


In [13]:
method = to_method_object('TF_IDF')
method.train(contexts_train, responses_train)
 # kpq: Top k candidates per query, for better computation.
good_candidates_index = method.sort_responses(test_x_text, candidates, min(args_kpq, len(candidates))) 
good_candidates = [[candidates[y] for y in x] for x in good_candidates_index]

good_candidates = save_data('backup/good_candidates.pkl', good_candidates)
good_candidates[1][:5]

Data has been saved to backup/good_candidates.pkl.


['using of is n-word and unacceptable in crazy race insults and not and people.',
 "it's of need is labels used discourse and you you of a for i mental away.",
 'racist not your use on to the about get a we labels?',
 'i the is no baiting is language and is who of the on can need you that is, comments. and most bigger and',
 'attacking there, as your and when you any our group over. who language we could use your polite amount you hateful']

## Module 3: Response Selection

In [ ]:
# METHODS = ['TF_IDF', 'USE_QA_SIM', 'USE_QA_MAP']

In [67]:
METHODS = ['TF_IDF']
for method_name in METHODS:
    print(method_name)
    method = to_method_object(method_name)
    method.train(contexts_train, responses_train)
    output = []
    for i, test_i in enumerate(tqdm(test_x_text)):
        predictions = method.rank_responses([test_i], good_candidates[i])
        output.append(good_candidates[i][predictions.item()])
    print(output)
print('*' * 80)
print(f'After filtering by LQ, there are {len(candidates)} candidates.\n')

TF_IDF


100%|██████████| 2923/2923 [00:04<00:00, 693.84it/s]

["I get you feel like he is a weak person, but it's unnecessary to use derrogatory terms about one's sexual orientation to make your point.", 'Not all African American men want to kill you, just like not all White males want to kill you. However, both happen.', 'You need to watch your language. This is not appropriate.', 'Apparently you do not understand what the word you are using means. It would be great if you would explain why you feel that way.', "I'm sure they are doing what they can but no need to use words like that.", 'that is not how to talk anyone!', 'You dont have to bring up his religion over this.', 'Not all African American men want to kill you, just like not all White males want to kill you. However, both happen.', 'The term "fa--ot" is offensive to many people and so should not be used in an illiterate political commentary.', "it's please please", "this term is because i'm", 'This post could be offensive to those with mental disabilities or jewish people.', "its offens

In [70]:
METHODS = ['USE_QA_SIM']
for method_name in METHODS:
    print(method_name)
    method = to_method_object(method_name)
    method.train(contexts_train, responses_train)
    output = []
    for i, test_i in enumerate(tqdm(test_x_text)):
        predictions = method.rank_responses([test_i], good_candidates[i])
        output.append(good_candidates[i][predictions.item()])
    print(output)
print('*' * 80)
print(f'After filtering by LQ, there are {len(candidates)} candidates.\n')

USE_QA_SIM


NameError: name 'vector_based' is not defined